In [2]:
import pandas as pd
import numpy as np

In [3]:
uid_train = pd.read_csv('../../trainData/uid_train.txt',sep='\t',header=None,names=('uid','label'))
voice_train = pd.read_csv('../../trainData/voice_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str})
sms_train = pd.read_csv('../../trainData/sms_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str})
wa_train = pd.read_csv('../../trainData/wa_train.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str})

/home/xuegsh/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
voice_test = pd.read_csv('../../testData_B/voice_test_b.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str})
sms_test = pd.read_csv('../../testData_B/sms_test_b.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str})
wa_test = pd.read_csv('../../testData_B/wa_test_b.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str})

In [5]:
uid_test = pd.DataFrame({'uid':pd.unique(wa_test['uid'])})
uid_test.to_csv('../data/uid_test_a.txt',index=None)

In [6]:
voice = pd.concat([voice_train,voice_test],axis=0)
sms = pd.concat([sms_train,sms_test],axis=0)
wa = pd.concat([wa_train,wa_test],axis=0)
uid = pd.concat([uid_train, uid_test], axis=0)

In [7]:
# 将数据中的时间都换算为秒
def time2Second(time):
    seconds = int(time[0:2])*24*3600 + int(time[2:4])*3600 + int(time[4:6])*60 + int(time[6:8])
    return seconds

voice['start_time'] = voice['start_time'].apply(time2Second)
voice['end_time'] = voice['end_time'].apply(time2Second)
sms['start_time'] = sms['start_time'].apply(time2Second)

In [8]:
# 计算每一次通话的时间
voice['time_last'] = voice['end_time'] - voice['start_time']

In [9]:
risk_uids_train = uid_train[uid_train['label'] == 1]['uid']
non_risk_uids_train = uid_train[uid_train['label'] == 0]['uid']


# 对通话记录构造特征

根据前面的数据探索，由于数据集大部分由类别数据或者离散型数据构成，所以首先针对通话数据按照uid分组统计其各项属性的数目，比如使用对端号码数量等等

In [46]:
# 计算每一次通话与上一次的时间间隔
from collections import defaultdict

def generate_timediff(group):
    time_diff = []
    last_row = None
    
    for i, row in group.iterrows():
        if last_row is None:
            last_row = row
            continue

        time_diff.append(row['start_time'] - last_row['end_time'])
        last_row = row            
    
    if len(time_diff) == 0:
        diff_mean = last_row['start_time']
        diff_std = 0
        diff_median = last_row['start_time']
        diff_zeros = 0
    else:
        diff_mean = np.mean(time_diff)
        diff_std = np.std(time_diff)
        diff_median = np.median(time_diff)
        time_diff = pd.Series(time_diff)
        diff_zeros = len(time_diff[time_diff < 15])  # 时间间隔少于15s的次数
    
    result = pd.Series({'time_diff_mean': diff_mean, 'time_diff_std': diff_std, 'time_diff_median': diff_median, 'time_diff_zeros': diff_zeros})
    
    return result
    

def formDiffDataFrame(df, prefix):
    df_result = pd.DataFrame()
    cols = ['time_diff_mean', 'time_diff_std', 'time_diff_median', 'time_diff_zeros']
    
    first_time = True
    for col in cols:
        t = df.unstack()[:][col].add_prefix(prefix + '_' + col + '_').reset_index()
        if first_time:
            df_result = t
            first_time = False
        else:
            df_result = pd.merge(df_result, t, how='left', on='uid')
    
    return df_result
    


In [11]:
# 对于每一个uid，根据in_out计算每一次通话（主叫和被叫）与上一次的时间间隔
temp = voice.sort_values(by='start_time').groupby(['uid', 'in_out']).apply(generate_timediff)
voice_timediff_by_in_out = formDiffDataFrame(temp, 'voice_in_out')

In [12]:
# 对于一个uid，计算每一次通话与上一次的时间间隔
temp = voice.sort_values(by='start_time').groupby(['uid']).apply(generate_timediff)
voice_timediff_by_uid = temp.add_prefix('voice_uid_').reset_index()

In [13]:
# 对于每一个uid，根据call_type计算每一次通话（本地，省内，省际，港澳台，国际）与上一次的时间间隔
temp = voice.sort_values(by='start_time').groupby(['uid', 'call_type']).apply(generate_timediff)
voice_timediff_by_call_type = formDiffDataFrame(temp, 'voice_call_type')

In [10]:
# 对于一个uid，opp_num的总数和unique数量
voice_opp_num = voice.groupby(['uid'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('voice_opp_num_').reset_index()
 
# 对于一个uid，opp_head的总数和unique数量
voice_opp_head = voice.groupby(['uid'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x))}).add_prefix('voice_opp_head_').reset_index()

# 对于一个uid每一种opp_len的数量
voice_opp_len = voice.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('voice_opp_len_').reset_index().fillna(0)

# 对于一个uid每一种call_type的数量
voice_call_type = voice.groupby(['uid','call_type'])['uid'].count().unstack().add_prefix('voice_call_type_').reset_index().fillna(0)

# 对于一个uid, 主叫和被叫的数量
voice_in_out = voice.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('voice_in_out_').reset_index().fillna(0)


/home/xuegsh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
/home/xuegsh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """


In [37]:
# 查看部分风险用户和非风险用户的数据是会否有明显差别
indices1 = risk_uids_train.head(10).values
indices2 = non_risk_uids_train.head(10).values

# build a DataFrame for the choosed indices
samples = pd.DataFrame(voice_time_all[voice_time_all['uid'].isin(indices1)])
samples1 = pd.DataFrame(voice_time_all[voice_time_all['uid'].isin(indices2)])

pd.concat([samples, samples1], axis=0)

,uid,voice_time_all
4090,u4100,990
4091,u4101,29632
4092,u4102,25605
4093,u4103,1605
4094,u4104,17228
4095,u4105,11361
4096,u4106,5423
4097,u4107,32260
4098,u4108,11683
4099,u4109,10935


In [35]:
# 对于一个uid，它的总通话时间，主叫时间，被叫时间（单位：秒）
voice_time_mean = voice.groupby(['uid', 'in_out'])['time_last'].mean().unstack().add_prefix('voice_mean_time_in_out_').reset_index().fillna(0)
voice_time_min = voice.groupby(['uid', 'in_out'])['time_last'].min().unstack().add_prefix('voice_min_time_in_out_').reset_index().fillna(0)
voice_time_max = voice.groupby(['uid', 'in_out'])['time_last'].max().unstack().add_prefix('voice_max_time_in_out_').reset_index().fillna(0)
voice_time_all = voice.groupby('uid')['time_last'].sum().reset_index().fillna(0)
voice_time_all.columns = ['uid', 'voice_time_all']


In [16]:
feat_voice = uid

features0 = [voice_opp_num, voice_opp_head, voice_opp_len, voice_call_type, voice_in_out, voice_time_mean,
            voice_time_min, voice_time_max, voice_time_all]
features1 = [voice_timediff_by_uid, voice_timediff_by_in_out, voice_timediff_by_call_type]

for feat in features0:
    feat_voice = pd.merge(feat_voice, feat, how='left', on='uid').fillna(0)

for feat in features1:
    feat_voice = pd.merge(feat_voice, feat, how='left', on='uid')  

feat_voice.to_csv('../myFeatures/feat_voice.csv')

In [17]:
feat_voice[feat_voice['uid'].isin(risk_uids_train)][['voice_opp_num_count', 'voice_time_all', 'voice_in_out_0', 'voice_in_out_1']]

,voice_opp_num_count,voice_time_all,voice_in_out_0,voice_in_out_1
4099,15.0,990.0,12.0,3.0
4100,215.0,29632.0,58.0,157.0
4101,673.0,25605.0,377.0,296.0
4102,17.0,1605.0,11.0,6.0
4103,226.0,17228.0,91.0,135.0
4104,126.0,11361.0,114.0,12.0
4105,52.0,5423.0,24.0,28.0
4106,784.0,32260.0,487.0,297.0
4107,66.0,11683.0,9.0,57.0
4108,135.0,10935.0,91.0,44.0


# 对短信记录构造特征

In [53]:
def generate_timediff_for_sms(group):       
    
    time_diff = np.ediff1d(group['start_time'])
    
    if len(time_diff) == 0:
        if (len(group['start_time']) == 1):
            diff_mean = group['start_time'].values[0]
            diff_std = 0
            diff_median = group['start_time'].values[0]
            diff_zeros = 0
    else:
        diff_mean = np.mean(time_diff)
        diff_std = np.std(time_diff)
        diff_median = np.median(time_diff)
        time_diff = pd.Series(time_diff)
        diff_zeros = len(time_diff[time_diff < 15])  # 时间间隔少于15s的次数
    
    result = pd.Series({'time_diff_mean': diff_mean, 'time_diff_std': diff_std, 'time_diff_median': diff_median, 'time_diff_zeros': diff_zeros})
    
    return result

In [54]:
# 对于一个uid，计算每次收到或发送短信与上一次的时间间隔
temp = sms.sort_values(by='start_time').groupby('uid').apply(generate_timediff_for_sms)
sms_timediff_by_uid = temp.add_prefix('sms_uid_').reset_index()

In [55]:
sms_timediff_by_uid.dtypes

uid                          object
sms_uid_time_diff_mean      float64
sms_uid_time_diff_median    float64
sms_uid_time_diff_std       float64
sms_uid_time_diff_zeros     float64
dtype: object

In [56]:
# 对于每一个uid，根据in_out计算每一次短信（发送和接收）与上一次的时间间隔
temp = sms.sort_values(by='start_time').groupby(['uid', 'in_out']).apply(generate_timediff_for_sms)
sms_timediff_by_in_out = formDiffDataFrame(temp, 'sms_in_out')

In [38]:
# 对于一个uid，opp_num的总数和unique数量
sms_opp_num = sms.groupby(['uid'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('sms_opp_num_').reset_index()

# 对于一个uid，opp_head的unique数量
sms_opp_head = sms.groupby(['uid'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x))}).add_prefix('sms_opp_head_').reset_index()

# 对于一个uid，opp_len的unique数量
sms_opp_len = sms.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('sms_opp_len_').reset_index().fillna(0)

# 对于一个uid，发出和收到短信的e数量
sms_in_out = sms.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('sms_in_out_').reset_index().fillna(0)


/home/xuegsh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
/home/xuegsh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """


In [58]:
# 查看部分风险用户和非风险用户的数据是会否有明显差别
indices1 = risk_uids_train.head(10).values
indices2 = non_risk_uids_train.head(10).values

# build a DataFrame for the choosed indices
samples = pd.DataFrame(sms_timediff_by_in_out[sms_timediff_by_in_out['uid'].isin(indices1)])
samples1 = pd.DataFrame(sms_timediff_by_in_out[sms_timediff_by_in_out['uid'].isin(indices2)])

pd.concat([samples, samples1], axis=0)

in_out,uid,sms_in_out_time_diff_mean_0,sms_in_out_time_diff_mean_1,sms_in_out_time_diff_std_0,sms_in_out_time_diff_std_1,sms_in_out_time_diff_median_0,sms_in_out_time_diff_median_1,sms_in_out_time_diff_zeros_0,sms_in_out_time_diff_zeros_1
4066,u4100,NaN,1.455265e+05,NaN,207689.383439,NaN,74332.0,NaN,10.0
4067,u4101,2.538102e+05,4.430449e+04,3.244220e+05,67395.575919,183373.0,2254.0,2.0,38.0
4068,u4102,7.464082e+04,NaN,1.007098e+05,NaN,27011.0,NaN,4.0,NaN
4069,u4103,1.293799e+06,1.128250e+05,0.000000e+00,134639.135828,1293799.0,64858.0,0.0,5.0
4070,u4104,3.805513e+05,2.077713e+04,3.777941e+05,29622.846636,322149.0,7249.0,2.0,58.0
4071,u4105,2.706810e+05,NaN,0.000000e+00,NaN,270681.0,NaN,0.0,NaN
4072,u4106,1.357751e+06,6.931706e+04,0.000000e+00,115640.566775,1357751.0,11415.0,0.0,19.0
4073,u4107,5.593820e+03,NaN,1.950622e+04,NaN,0.0,NaN,376.0,NaN
4074,u4108,9.106982e+05,3.674084e+05,1.533856e+06,716670.644766,37960.0,12954.0,2.0,4.0
4075,u4109,5.369885e+04,3.321115e+05,1.371518e+05,352129.059659,305.0,179814.5,4.0,2.0


In [22]:
feat_sms = uid

features0 = [sms_opp_num, sms_opp_head, sms_opp_len, sms_in_out]
features1 = [sms_timediff_by_uid, sms_timediff_by_in_out]

for feat in features0:
    feat_sms = pd.merge(feat_sms, feat, how='left', on='uid').fillna(0)

for feat in features1:
    feat_sms = pd.merge(feat_sms, feat, how='left', on='uid')
    
feat_sms.to_csv('../myFeatures/feat_sms.csv')

# 对上网记录构造特征

In [23]:
# 对于一个uid，wa_name的总数和unique数量(并没有包含值为空串的wa_name)
wa_name = wa.groupby(['uid'])['wa_name'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('wa_name_').reset_index()

# 对于一个uid，当天访问该网站次数visit_cnt的'std','max','min','median','mean','sum'
visit_cnt = wa.groupby(['uid'])['visit_cnt'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_cnt_').reset_index()

# 对于一个uid，当天访问该网站总时长visit_dura的'std','max','min','median','mean','sum'
visit_dura = wa.groupby(['uid'])['visit_dura'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_dura_').reset_index()

# 对于一个uid，当天访问该网站总上行流量up_flow的'std','max','min','median','mean','sum'
up_flow = wa.groupby(['uid'])['up_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_up_flow_').reset_index()

# 对于一个uid，当天访问该网站总下行流量down_flow的'std','max','min','median','mean','sum'
down_flow = wa.groupby(['uid'])['down_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_down_flow_').reset_index()

# 对于一个uid，这45天中上网的天数
visit_days_count = wa.groupby(['uid'])['date'].apply(lambda x: len(x.unique())).add_prefix('wa_visit_days_count').reset_index()

/home/xuegsh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


In [24]:
feat_wa = uid

features = [wa_name, visit_cnt, visit_dura, up_flow, down_flow, visit_days_count]

for feat in features:
    feat_wa = pd.merge(feat_wa, feat, how='left', on='uid').fillna(0)

feat_wa.to_csv('../myFeatures/feat_wa.csv')

# 连接所有特征

In [67]:
feat_wa.columns

Index(['label', 'uid', 'wa_name_unique_count', 'wa_name_count',
       'wa_visit_cnt_std', 'wa_visit_cnt_max', 'wa_visit_cnt_min',
       'wa_visit_cnt_median', 'wa_visit_cnt_mean', 'wa_visit_cnt_sum',
       'wa_visit_dura_std', 'wa_visit_dura_max', 'wa_visit_dura_min',
       'wa_visit_dura_median', 'wa_visit_dura_mean', 'wa_visit_dura_sum',
       'wa_up_flow_std', 'wa_up_flow_max', 'wa_up_flow_min',
       'wa_up_flow_median', 'wa_up_flow_mean', 'wa_up_flow_sum',
       'wa_down_flow_std', 'wa_down_flow_max', 'wa_down_flow_min',
       'wa_down_flow_median', 'wa_down_flow_mean', 'wa_down_flow_sum', 'date'],
      dtype='object')

In [25]:
feat_all = feat_voice
feat_all = pd.merge(feat_all, feat_sms.drop(, how='left', on='uid')
feat_all = pd.merge(feat_all, feat_wa, how='left', on='uid')


In [26]:
feat_voice.describe()

,label,voice_opp_num_unique_count,voice_opp_num_count,voice_opp_head_unique_count,voice_opp_len_3,voice_opp_len_5,voice_opp_len_6,voice_opp_len_7,voice_opp_len_8,voice_opp_len_9,...,voice_call_type_time_diff_median_1,voice_call_type_time_diff_median_2,voice_call_type_time_diff_median_3,voice_call_type_time_diff_median_4,voice_call_type_time_diff_median_5,voice_call_type_time_diff_zeros_1,voice_call_type_time_diff_zeros_2,voice_call_type_time_diff_zeros_3,voice_call_type_time_diff_zeros_4,voice_call_type_time_diff_zeros_5
count,7999.000000,7999.000000,7999.000000,7999.000000,7999.000000,7999.000000,7999.000000,7999.000000,7999.000000,7999.000000,...,7.893000e+03,3.970000e+03,4.608000e+03,6.000000e+00,3.700000e+01,7893.000000,3970.000000,4608.000000,6.0,37.0
mean,0.112514,48.666708,220.071634,17.155394,0.064383,1.597450,0.062258,0.311289,1.179272,0.171771,...,1.317978e+05,5.691964e+05,5.249685e+05,9.898199e+05,6.483988e+05,4.974154,0.223426,0.325738,0.0,0.0
std,0.316018,145.778259,388.442683,11.985497,0.490704,4.125528,0.650177,1.873630,4.446100,0.897050,...,4.466774e+05,9.541908e+05,9.286409e+05,1.345461e+06,1.041526e+06,24.873025,1.384762,1.739995,0.0,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,8.000000e+00,0.000000e+00,-1.800000e+01,2.315000e+02,1.350000e+02,0.000000,0.000000,0.000000,0.0,0.0
25%,0.000000,9.000000,27.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.595000e+03,1.545012e+04,2.155812e+04,8.787100e+04,8.869000e+03,0.000000,0.000000,0.000000,0.0,0.0
50%,0.000000,22.000000,91.000000,15.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,1.064350e+04,9.451875e+04,8.958500e+04,2.693970e+05,1.017240e+05,0.000000,0.000000,0.000000,0.0,0.0
75%,0.000000,50.000000,239.000000,25.000000,0.000000,2.000000,0.000000,0.000000,1.000000,0.000000,...,5.699600e+04,6.124731e+05,4.657524e+05,1.722269e+06,6.414760e+05,2.000000,0.000000,0.000000,0.0,0.0
max,1.000000,8038.000000,8129.000000,86.000000,29.000000,223.000000,49.000000,67.000000,143.000000,35.000000,...,3.961498e+06,3.966260e+06,3.967992e+06,3.165972e+06,3.860916e+06,636.000000,64.000000,58.000000,0.0,0.0


由各项统计的最大值与中位值，75%值的比较可以看到, 每一项多少都存在一些异常数值，或许可以作为异常行为进行观察

In [64]:
feat_all.columns

Index(['label_x', 'uid', 'voice_opp_num_unique_count', 'voice_opp_num_count',
       'voice_opp_head_unique_count', 'voice_opp_len_3', 'voice_opp_len_5',
       'voice_opp_len_6', 'voice_opp_len_7', 'voice_opp_len_8',
       ...
       'wa_up_flow_median', 'wa_up_flow_mean', 'wa_up_flow_sum',
       'wa_down_flow_std', 'wa_down_flow_max', 'wa_down_flow_min',
       'wa_down_flow_median', 'wa_down_flow_mean', 'wa_down_flow_sum', 'date'],
      dtype='object', length=136)

In [27]:
train_features = feat_all[feat_all['uid'].isin(uid_train['uid'])]


In [28]:
train_features.shape

(4999, 136)

In [29]:
test_features = feat_all[feat_all['uid'].isin(uid_test['uid'])]

In [30]:
test_features.shape

(3000, 136)

In [31]:
train_features.to_csv('../myFeatures/train_features.csv',index=None)
test_features.to_csv('../myFeatures/test_features.csv',index=None)